In [6]:
# import

# math
from math import sqrt
from numpy import concatenate
from numpy import mean

# graph and table
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import pandas as pd

# other
import random
import glob

# machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import holidays

# option
pd.options.mode.chained_assignment = None 

In [7]:
# PATHS

#Den Bosch flow
path = "../data/sewer_data/data_pump/RG8150/RG8150/"
path1 = "../sewer_data_db/data_wwtp_flow/RG1876_flow/"
path2 = "../sewer_data_db/data_wwtp_flow/RG1882_flow/"


#Bokhoven level
path3 = "../data/sewer_data/data_pump/RG8180_L0/"
#Bokhoven flow
path4 = "../data/sewer_data/data_pump/RG8180_Q0/"


#Haarsteeg level
path5 = "../data/sewer_data/data_pump/rg8170_N99/"
#Haarsteeg flow
path6 = "../data/sewer_data/data_pump/rg8170_99/"


#Helftheuvelweg level column 003 Helftheuvelweg *.csv
path7 = "../data/sewer_data_db/data_pump_level/"
#Helftheuvelweg flow 
path8 = "../data/sewer_data_db/data_pump_flow/1210FIT301_99/"


#Engelerschans level column “004 Engelerschans” *.csv
path9 = "../data/sewer_data_db/data_pump_level/"
#Engelerschans flow + Haarsteeg + Bokhoven, therefore substract for only Engeleschans
path10 = "../data/sewer_data_db/data_pump_flow/1210FIT201_99/"


#Maaspoort level Column: “006 Maaspoort” *.csv 
path11 = "../data/sewer_data_db/data_pump_level/"
#Maasport flow + Rompert
path12= "../data/sewer_data_db/data_pump_flow/1210FIT501_99/"


#Oude Engelenseweg level Column: “002 Oude Engelenseweg” *.csv
path13 = "../data/sewer_data_db/data_pump_level/"
#Oude Engelenseweg flow
path14 = "../data/sewer_data_db/data_pump_flow/1210FIT401_94/"


#De Rompert level Column: “005 de Rompert” *.csv
path15 = "../data/sewer_data_db/data_pump_level/"
#De Rompert flow + Maasport
path16 = "../data/sewer_data_db/data_pump_flow/1210FIT501_99/"

#Location linkage
path_linkinfo = "../data/sewer_model"
path_rain = "../data/sewer_data/rain_timeseries"

#Missing Engelerschans (in map also)
station_names = ["Haarsteeg", "Bokhoven", "Hertogenbosch (Helftheuvelweg)",
                 "Hertogenbosch (Rompert)", "Hertogenbosch (Oude Engelenseweg)",
                 "Hertogenbosch (Maasport)"]

flow_paths = [path1, path2, path4, path6, path8, path10, path12, path14, path16]
level_paths = [path3, path5, path7, path9, path11, path13, path15]

In [8]:
def path_bag(flow, level, station_names):
    #station_to_flow = dict()
    #station_to_level = dict()
    return None


def streets_rain(station_names, path_linkinfo, path_rain):
    
    link = pd.read_excel(path_linkinfo+
                   "/20180717_dump riodat rioleringsdeelgebieden_matched_to_rainfall_locations.xlsx",
                   header = 9)
    
    rain = pd.concat([pd.read_csv(file, header = 2) for file in glob.glob(path_rain+"/*.*")], ignore_index = True)
    
    #Street names by stations
    streets = [list(link[(link["Naam kern"] == name)]["Naam / lokatie"]) for name in station_names]

    #Streets that are not found in rainfall data belonging to Hertogenbosch (Oude Engelenseweg)
    excl = ['Pettelaarpark', 'Geb. 16 Paleiskwartier']

    for s in streets:
        try:
            for i in excl:
                s.remove(i)
        except ValueError:
            pass


    #All the rain for the streets for the pump stations in order of station_names and the streets per station
    #can be found in streets nested list in the same order
    station_to_rain = rain[["Begin", "Eind"] + [i for sl in streets for i in sl]]

    station_rain = pd.DataFrame()
    for i in range(len(station_names)):
        station_rain[station_names[i]] = rain[streets[i]].sum(1)

    station_rain["Begin"] =  station_to_rain["Begin"]
    station_rain["End"] = station_to_rain["Eind"]
    station_rain["Begin"] = pd.to_datetime(station_rain["Begin"])
    station_rain["End"] = pd.to_datetime(station_rain["End"])
    station_rain["date"] = station_rain["Begin"].dt.date
    return station_rain.sort_values(by=["Begin"])

def labeler3(dayrain, previousdayrain):
    """
    Classifies a whole day as dry if previous OR the same day has less than 0.05
    """
    if dayrain <= 0.05 or previousdayrain <= 0.05:
        return 0
    else:
        return 1
        
def rainyday(df, station_names):
    """
    Returns a list with DFs per station for dry day classification
    """
    df_rainyday = df.groupby(df["Begin"].dt.date).sum().reset_index(drop=False)
    
    events_df = []
    
    for i in station_names:
        df_relevant = df_rainyday[[i, "Begin"]]
        df_relevant["previousday"] = df_relevant[i].shift(-1)
        df_relevant["dryday"] = df_relevant.apply(lambda x: labeler3(x[i], x["previousday"]), axis=1)
        events_df.append(df_relevant)
        
    return events_df
    

def labeler2(rain):
    """
    Classifies an hour as dry if there hasn't been rain previous n days days
    """
    threshold = 0.05*(5/8)
    if rain >= 0 and rain <= threshold:
        return 0
    else:
        return 1
    
def last_n_cumsum(n, name, df):
    B = []
    i =0
    n_lim = n
    station = list(df[name])
    while i<len(station):
        if i<n_lim:
            B.append(sum(station[0:i]))
        if i>=n_lim:
            B.append(sum(station[i-n_lim:i]))
        i=i+1
    return B

def binary_rain(station_names, df, n):
    
    binary_rain_df = []
    df = df.sort_values(by="Begin")
    df = df.set_index("Begin", drop = False).resample("60Min").sum()
    df = df.reset_index(drop=False)
    
    for i in station_names:
        df_relevant = df[[i, "Begin"]]
        df_relevant["cumsum_previous_15"] = last_n_cumsum(n, i, df)
        df_relevant["rain_-15_class"] = df_relevant.apply(lambda x: labeler2(x["cumsum_previous_15"]), axis=1)
        binary_rain_df.append(df_relevant)
    return binary_rain_df

# Here n = 15
#events_df = rain_events2(station_names, df, 15)


counter = 0
def labeler(rain, previous_rain):
    """
    Labels rain events (events are defined as consecutive measurements where there is no stop of rain)
    (Previous definition)
    """
    global counter
    if rain == 0:
        return "None"
    else:
        if previous_rain == 0:
            counter += 1
            return counter
        if previous_rain != 0:
            return counter
        
def rain_events(station_names, df):
    events_df = []
    df = df.sort_values(by="Begin")
    
    for i in station_names:
        df_relevant = df[[i, "Begin", "End"]]
        df_relevant["instr"] = df_relevant[i].shift()
        df_relevant["rain_event_ID"] = df_relevant.apply(lambda x: labeler(x[i], x["instr"]), axis=1)
        df_relevant.drop(columns=["instr"], inplace=True)
        events_df.append(df_relevant)
    return events_df

#events_df = rain_events(station_names, df)

def group_by_event(events_df, station_name, rain_event_col, k):
    df = events_df[k]
    
    #Filtering rain events where there was no rain
    df = df[df[rain_event_col] != "None"]
    
    #should be elsewhere
    df["End"] = pd.to_datetime(df["End"])
    #
    
    #Converting duration of individual measurements to minutes
    df["duration"] = (df["End"] - df["Begin"]).dt.total_seconds()/60
    
    #Filter measurements that last more than 1 week (there are errors in data)
    df = df[(df["duration"] <= 10080) & (df["duration"] >= 0)]
    
    #Group by rain event ID and sum the durations and rain measurements
    df = df[["duration", station_name, rain_event_col]].groupby(rain_event_col).sum()
    return df


def bound_dates(df1, df2, df1_datecol, df2_datecol):
    
    # Making sure both have the same range of dates
    df1[df1_datecol] = pd.to_datetime(df1[df1_datecol])
    df2[df2_datecol] = pd.to_datetime(df2[df2_datecol])
    
    df1 = pd.merge(left=df1, left_on=df1_datecol,
         right=df2, right_on=df2_datecol)
    #d2 = pd.merge(left=df2, left_on=df2_datecol,
    #     right=df1, right_on=df1_datecol)
    df1.drop(columns=[df2_datecol])
    return df1

def hourly_conversion(path, mean=bool):
    """
    Converts data to hourly format
    """
    
    df = pd.concat([pd.read_csv(file) for file in glob.glob(path+"/*.*")], ignore_index = True)
    df = df[["datumBeginMeting", "datumEindeMeting", "hstWaarde"]].sort_values(by='datumBeginMeting')
    df["datumBeginMeting"] = pd.to_datetime(df["datumBeginMeting"])
    df["datumEindeMeting"] = pd.to_datetime(df["datumEindeMeting"])
    if mean == True:
        df = df.set_index("datumBeginMeting", drop = False).resample("60Min").mean()
    else:
        df = df.set_index("datumBeginMeting", drop = False).resample("60Min").sum()
   
    df = df.reset_index()
    return df
    
rain_df = streets_rain(station_names, path_linkinfo, path_rain)
hourly_rain_classified = binary_rain(station_names, rain_df, n=15)



flow_bokhoven = hourly_conversion(path4, mean = False)
flow_bokhoven  = bound_dates(flow_bokhoven, hourly_rain_classified[1], "datumBeginMeting", "Begin")

# flow_bokhoven.to_csv('../data/Bokhoven_rain.csv')

In [9]:
# Rain in each station df
rain_df = streets_rain(station_names, path_linkinfo, path_rain)

# List of dfs with each station's hourly rain classification in order of station_names list,
# with n = 15. (n=15 means rain_-15_class is "0" if no rain prior 15 hours and "1" otherwise)
hourly_rain_classified = binary_rain(station_names, rain_df, n=15)

# Level and flow of Haarsteeg per hour (shapes match here, have to check if they do on other files,
# otherwise bound dates like in line 86)
level_haarsteeg = hourly_conversion(path5, mean = True)
flow_haarsteeg = hourly_conversion(path6, mean = False)

# Returns merged dataframe with the timestamps present in both dfs
flow_haarsteeg  = bound_dates(flow_haarsteeg, hourly_rain_classified[0], "datumBeginMeting", "Begin")

nl_holidays = holidays.CountryHoliday('NL')

def binary_holidays(country_holidays, dates):
    holiday = []
    for i in dates:
        if i in nl_holidays:
            holiday.append(1)
        else:
            holiday.append(0)
    return holiday
    


def feature_setup(df_flow, df_level, country_holidays):
    
    dates = df_flow["Begin"]
    flow = df_flow["hstWaarde"]
    rained_n_class = df_flow["rain_-15_class"]
    rain = df_flow["Haarsteeg"]
    #cumsum_previous_n = df_flow["cumsum_previous_15"]
    level = df_level["hstWaarde"]
    holidays = binary_holidays(nl_holidays, dates)
    
    features = pd.DataFrame()
    
    features["day_ofthe_month"] = dates.dt.day
    features["hour"] = dates.dt.hour
    features["day_ofthe_year"] = dates.dt.dayofyear
    features["day_ofthe_week"] = dates.dt.dayofweek
    features["holiday"] = holidays
    features["flow"] = flow
    
    # Add feature of amount of rain some timestamps before or during this hour (5 minute stamps)
    features["rain_hour"] = rain
    
    # Add a feature of level at previous timestamps before or during this hour (5 minute stamps)
    features["level"] = level
    
    #Not actual features, but columns by which we will filter prediction procedures
    #(Be sure to remove them before fitting)
    features["rain_N_ago"] = rained_n_class
    features["dates"] = dates
    

    return features

df_features = feature_setup(flow_haarsteeg, level_haarsteeg, nl_holidays)


def mse(d):
    """Mean Squared Error"""
    return mean(d * d) 

# def random_forest(features):
#     features = features[df_features['rain_N_ago'] == 0]
    
#     all_days = set(features["dates"].dt.date)
#     test_days = random.sample(all_days, 120)
#     training_days = [x for x in all_days if x not in test_days]
    
#     training = features[features["dates"].dt.date.isin(training_days)]
#     training_X = training.drop(columns = ["flow", "dates"])
#     training_Y = training["flow"]
    
#     testing = features[features["dates"].dt.date.isin(test_days)]
#     testing_X = testing.drop(columns = ["flow", "dates"])
#     testing_Y = testing["flow"]
    
#     rf = RandomForestRegressor(n_estimators = 1000)
#     rf.fit(training_X, training_Y)
#     prediction = rf.predict(testing_X)
#     error = testing_Y - prediction
#     return print(rf.feature_importances_, sqrt(mse(error)))
    

# random_forest(df_features)

In [10]:
combined_rain_pump = pd.read_pickle("../data/combined_data/combined_rain_pump_1hour_mean.pickle")

In [11]:
combined_rain_pump.head()

,Bokhoven,Engelerschans,Helftheuvelweg,Maaspoort,Oude Engelenseweg,de Rompert,Haarsteeg,RG8180_flow,RG8180_level,RG8170_flow,RG8170_level
datumBeginMeting,,,,,,,,,,,
2017-12-31 23:00:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN
2018-01-01 00:00:00,0.0264,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,20.000000,1.028333,3657.320000,-0.308333
2018-01-01 01:00:00,0.0224,0.0119,0.0198,0.0038,0.0000,0.0006,0.0636,15.666667,0.376667,3356.713333,-0.846667
2018-01-01 02:00:00,0.0675,0.0057,0.0069,0.0374,0.0265,0.0311,0.0172,0.000000,0.160000,2755.048333,-1.198333
2018-01-01 03:00:00,0.0273,0.0032,0.0078,0.0262,0.0014,0.0377,0.0215,4.666667,0.061667,2394.443333,-1.273333


In [9]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
series_to_supervised(combined_rain_pump)